### Libraries

In [76]:
%%capture
%reset -f  
'generic imports'
import pandas as pd
import json
import time 
from IPython.display import display, clear_output
from numba import cuda  

'data augmentation imports'
from realtabformer import REaLTabFormer  # https://github.com/worldbank/REaLTabFormer

### GPU

In [77]:
# Get the current CUDA context and GPU information
ctx = cuda.current_context()
meminfo = ctx.get_memory_info()
device_name = cuda.gpus[0].name.decode()
cudnn_version = cuda.cudadrv.driver.get_version()

# Print the GPU information
print(f'GPU: {device_name}\nAvailable GPU memory: {meminfo[0] / 1024**3:.1f} GB\ncuDNN version: {cudnn_version}')

GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Available GPU memory: 7.5 GB
cuDNN version: (12, 2)


### Loading Data

In [78]:
df_train = pd.read_csv('../data/EdgeIIot_train_100k.csv', low_memory=False)

#### Attack types to be augmented

In [79]:
# Counts how many rows of each Attack_type are missing (100.000)
counts_dict = {}
for attack in df_train["Attack_type"].unique():
    counts_dict[attack] = 100000 - df_train[df_train["Attack_type"] == attack].shape[0]

print("Rows to be augmented by attack type:\n")
for key, value in sorted(counts_dict.items(), key=lambda item: item[1], reverse=True):
    print(f"{key:<22} {value:>10}")

Rows to be augmented by attack type:

MITM                        99718
Fingerprinting              99293
Ransomware                  92237
XSS                         87969
Port_Scanning               84085
Backdoor                    80756
Uploading                   70466
DDoS_HTTP                   61084
Password                    60175
DDoS_TCP                    59947
Vulnerability_scanner       59914
SQL_injection               59245
DDoS_ICMP                   45562
DDoS_UDP                     3034
Normal                          0


### Discard Redudant Data

In [82]:
# Since 'Normal' Attack_type already has 100.000 rows, we do not consider it for augmentation
df_attack = df_train[df_train["Attack_type"] != "Normal"]

# Drop the Attack_label because it can be reconstructed from the Attack_type after augmentation
df_attack = df_attack.drop(columns=["Attack_label"])

### Data type conversion (to save space)

In [26]:
# calculate the size of df_attack before
size_before = df_attack.memory_usage(deep=True).sum() / 1024**2

# select only the columns that are not object type
df_numeric = df_attack.select_dtypes(exclude=["object"])

# convert all numeric columns to unsigned integer type
df_numeric = df_numeric.apply(pd.to_numeric, downcast="unsigned")

# print df_numeric data types before and after conversion side by side
print(pd.concat([df_attack.dtypes, df_numeric.dtypes], axis=1,keys=["Before", "After"]))

# drop all numeric columns from df_attack
df_attack = df_attack.drop(columns=df_numeric.columns)

# concatenate df_attack and newly convertes df_numeric columns
df_attack = pd.concat([df_attack, df_numeric], axis=1)

# calculate the size of df_attack after conversion
size_after = df_attack.memory_usage(deep=True).sum() / 1024**2

# print size before and after conversion
print(f"\nSize of df_attack before data conversion: {size_before:.2f} MBytes, after: {size_after:.2f} MBytes")


/home/tiagociic/Projectos/inovmineral/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:375: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)


                            Before    After
arp.opcode                 float64    uint8
arp.hw.size                float64    uint8
icmp.checksum              float64   uint16
icmp.seq_le                float64   uint16
icmp.unused                float64    uint8
http.content_length        float64   uint32
http.request.method         object      NaN
http.referer                object      NaN
http.request.version        object      NaN
http.response              float64    uint8
http.tls_port              float64    uint8
tcp.ack                    float64   uint32
tcp.ack_raw                float64   uint32
tcp.checksum               float64   uint16
tcp.connection.fin         float64    uint8
tcp.connection.rst         float64    uint8
tcp.connection.syn         float64    uint8
tcp.connection.synack      float64    uint8
tcp.flags                  float64    uint8
tcp.flags.ack              float64    uint8
tcp.len                    float64   uint16
tcp.seq                    float

In [ ]:
# print head of df_attack
df_attack.head(10).style.set_properties(**{'text-align': 'left'})

,http.request.method,http.referer,http.request.version,dns.qry.name.len,mqtt.conack.flags,mqtt.protoname,mqtt.topic,Attack_type,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.qu,dns.qry.type,dns.retransmission,dns.retransmit_request,dns.retransmit_request_in,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msg_decoded_as,mqtt.msgtype,mqtt.proto_len,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1622034,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0.0,0,0.0,0.0,0.0,0.0,Password,0,0,0,0,0,0,0,0,303,3453508832,24759,0,0,0,0,16,1,0,262.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,2594368,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1871277,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,0,0.0,0,0.0,0.0,0.0,0.0,Password,0,0,0,0,0,0,0,0,1,1080021225,15803,0,0,0,0,16,1,0,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_TCP,0,0,0,0,0,0,0,0,2140609672,2140609672,20485,0,0,1,0,2,0,120,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Backdoor,0,0,0,0,0,0,0,0,1,1453306566,6843,0,0,0,0,16,1,1448,1463329.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29,0,0.0,0,0.0,0.0,0.0,0.0,Password,0,0,0,0,0,0,0,0,0,0,16528,0,0,1,0,2,0,0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,57101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,10834,0,655220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1510333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# separate df_attack by Attack_type into 2 dataframes with similar number of lines
df_attack_1 = df_attack[df_attack["Attack_type"].isin(["Backdoor", 
                                                       "DDoS_HTTP", 
                                                       "DDoS_ICMP", 
                                                       "DDoS_TCP", 
                                                       "DDoS_UDP", 
                                                       "Password"])]

df_attack_2 = df_attack[df_attack["Attack_type"].isin(["Port_Scanning", 
                                                       "SQL_injection", 
                                                       "Uploading", 
                                                       "Vulnerability_scanner", 
                                                       "Ransomware",
                                                       "XSS", 
                                                       "Fingerprinting", 
                                                       "MITM"])]
print("-----------------------")
print("df_attack_1")
# print number of lines per Attack_type for each dataframe and total number of lines for each dataframe
display(df_attack_1['Attack_type'].value_counts().to_frame().style.set_properties(**{'text-align': 'left'}))
# print sum of lines for df_attack_1
print(f"Number of lines: {df_attack_1['Attack_type'].value_counts().sum()}")
print(f"\nSize: {df_attack_1.memory_usage(deep=True).sum() / 1024**2:.2f} MBytes")
print("-----------------------\n")
print("df_attack_2")
# print number of lines per Attack_type for each dataframe and total number of lines for each dataframe
display(df_attack_2['Attack_type'].value_counts().to_frame().style.set_properties(**{'text-align': 'left'}))
# print sum of lines for df_attack_2
print(f"Number of lines: {df_attack_2['Attack_type'].value_counts().sum()}")
print(f"Size: {df_attack_2.memory_usage(deep=True).sum() / 1024**2:.2f} MBytes")
print("-----------------------")
      


-----------------------
df_attack_1


,count
Attack_type,
DDoS_UDP,96966
DDoS_ICMP,54438
DDoS_TCP,40053
Password,39825
DDoS_HTTP,38916
Backdoor,19244


Number of lines: 289442

Size: 154.04 MBytes
-----------------------

df_attack_2


,count
Attack_type,
SQL_injection,40755
Vulnerability_scanner,40086
Uploading,29534
Port_Scanning,15915
XSS,12031
Ransomware,7763
Fingerprinting,707
MITM,282


Number of lines: 147073
Size: 78.81 MBytes
-----------------------


# Training df_attack_1 classes model

In [ ]:
# REalTabFormer instation: non-relational data variant
rtf_model_df1 = REaLTabFormer(model_type="tabular",
                          batch_size=1,
                          gradient_accumulation_steps=1,
                          checkpoints_dir = "rtf_checkpoint_df_attack_1",
                          logging_steps=100,
                          random_state= 42,
                          epochs= 10,
                          train_size = 0.8,                    # 80% of the data for training, 20% for validation
                          numeric_max_len= 12
                          )

# Note: number of bootstrap samples has influnce in required RAM memory, before optimization

In [ ]:
# train df_attack_1 model
rtf_model_df1.fit(df_attack_1)
# save model
rtf_model_df1.save("rtf_models/")

# Training df_attack_2 classes model

In [ ]:
rtf_model_df2 = REaLTabFormer(model_type="tabular",
                          batch_size=1,
                          gradient_accumulation_steps=1,
                          checkpoints_dir = "rtf_checkpoint_df_attack_2_v2",
                          logging_steps=100,
                          random_state= config['random_state'],
                          epochs= 10,
                          train_size = 0.8,                    # 80% of the data for training, 20% for validation
                          numeric_max_len= 12
                          )

In [ ]:
# train df_attack_2 model
rtf_model_df2.fit(df_attack_2)
# save model
rtf_model_df2.save("rtf_models/")

Computing the sensitivity threshold...
Using parallel computation!!!


/home/tiagociic/Projectos/inovmineral/lib/python3.11/site-packages/realtabformer/realtabformer.py:534: UserWarning: The frac (0.165) set results to a sample larger than                     frac_max_data=10000. Setting frac to 0.06799344543186037.
  warnings.warn(
/home/tiagociic/Projectos/inovmineral/lib/python3.11/site-packages/realtabformer/realtabformer.py:568: UserWarning: Duplicate rate (0.0) in the data is zero. The `qt_interval` will be set                     to qt_interval_unique=100.
  warnings.warn(


Bootstrap round:   0%|          | 0/500 [00:00<?, ?it/s]

Sensitivity threshold summary:
count    500.000000
mean       0.000020
std        0.001220
min       -0.003965
25%       -0.000775
50%       -0.000038
75%        0.000776
max        0.003186
dtype: float64
Sensitivity threshold: 0.0020209899999999994 qt_max: 0.05


Map:   0%|          | 0/147073 [00:00<?, ? examples/s]

...

{'eval_loss': 0.47221875190734863, 'eval_runtime': 90.6136, 'eval_samples_per_second': 324.62, 'eval_steps_per_second': 324.62, 'epoch': 0.04}
{'loss': 0.4811, 'learning_rate': 4.977902055108875e-05, 'epoch': 0.04}


  0%|          | 0/29415 [00:00<?, ?it/s]

{'eval_loss': 0.47343870997428894, 'eval_runtime': 90.5622, 'eval_samples_per_second': 324.804, 'eval_steps_per_second': 324.804, 'epoch': 0.04}
{'train_runtime': 657.5831, 'train_samples_per_second': 1789.249, 'train_steps_per_second': 1789.249, 'train_loss': 0.06301491590646598, 'epoch': 0.04}


  0%|          | 0/30000 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 30080 samples generated. Sampling efficiency is: 100.0000%
Saving not-best model...
Critic round: 10,                     sensitivity_threshold: 0.0020209899999999994,                         val_sensitivity: 0.00399436,                             val_sensitivities: [0.0036542, 0.0032517999999999996, 0.0028190000000000003, 0.0022500000000000003, 0.003990800000000001, 0.005039199999999999, 0.005099600000000001, 0.003762600000000001, 0.0034178000000000004, 0.0046236, 0.0044816, 0.0031036, 0.0047409999999999996, 0.005065000000000001, 0.0046156]
Copying artefacts from: best-disc-model
Copying artefacts from: mean-best-disc-model
Copying artefacts from: not-best-disc-model
Copying artefacts from: last-epoch-model


# Data Generation

In [ ]:
# load best df_attack_1 model (id000016979230192943532032)
rtf_model_df1 = REaLTabFormer.load_from_dir(path="rtf_models/id000016979230192943532032")

In [ ]:
def sample_by_category(model, category_dict, columns):
    """
    Samples from the model by category.
    :param model: the model to sample from
    :param category_dict: a dictionary with the number of samples to generate for each category
    :param columns: the columns to include in the output dataframe
    :return: a dataframe with the samples
    """
    sampled_df = pd.DataFrame(columns=columns)
    for category, n_samples in category_dict.items():
        start_time = time.time()
        while len(sampled_df[sampled_df['Attack_type'] == category]) < n_samples:
            
            samples = model.sample(n_samples=20000)
            # select only the samples with the desired category
            category_samples = samples[samples['Attack_type'] == category]
            # discard the excess samples if necessary
            if len(category_samples) > n_samples:
                category_samples = category_samples.sample(n=n_samples)
            # add the selected samples to the sampled_df dataframe
            sampled_df = pd.concat([sampled_df, category_samples], ignore_index=True)
            
            elapsed_time = time.time() - start_time
            if elapsed_time > 5:
                clear_output(wait=True)
                print(f"{len(sampled_df[sampled_df['Attack_type'] == category])}/{n_samples}", end="\r")
                start_time = time.time()
    return sampled_df

In [ ]:
# Dictionary with the number of samples to be generated with rtf_model_df1
counts_df1= {'DDoS_UDP': 3034,
             'DDoS_HTTP':61084,
             'Password': 60175,
             'DDoS_TCP': 59947,
             'Backdoor': 80756,
             'DDoS_ICMP': 45562}

In [ ]:
# Sample by category for df_attack_1
RTB_dataset_df1 = sample_by_category(rtf_model_df1, counts_df1, df_train.columns)

  0%|          | 0/10000 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 10112 samples generated. Sampling efficiency is: 100.0000%


In [ ]:
# Save RTB_dataset_df1 to csv file
RTB_dataset_df1.to_csv("RTB_dataset_df1.csv", index=False)

In [ ]:
# Dictionary with the number of samples to be generated with rtf_model_df2
counts_df2= {"Port_Scanning": 84085, 
             "SQL_injection":59245 , 
             "Uploading":70466, 
             "Vulnerability_scanner":59914, 
             "Ransomware":92237, 
             "Fingerprinting":99293,
             "XSS": 87969,
             "MITM":99718}

In [ ]:
# Load best df_attack_2 model (id000016979230192943532032)
rtf_model_df2 = REaLTabFormer.load_from_dir(path="rtf_models/id000016980601259525519360")

In [ ]:
# TESTE

# counts_df2= {"XSS": 87969}
# rtf_model_df2 = REaLTabFormer.load_from_dir(path="rtf_models/id000016982514417271404544")  

# RTB_dataset_df2 = sample_by_category(rtf_model_df2, 
#                                      counts_df2, 
#                                      df_train.columns)

# save RTB_dataset_df2 to csv file
# RTB_dataset_df2.to_csv("RTB_dataset_df2_XSS.csv", index=False)

# Augmented data verification and concatenation

In [85]:
# Load RTB_dataset_df1 and RTB_dataset_df2
RTB_dataset_df1 = pd.read_csv("../data/RTB_dataset_df1.csv", low_memory=False)
RTB_dataset_df2 = pd.read_csv("../data/RTB_dataset_df2_v2.csv", low_memory=False)

In [86]:
# Attack type per dataset
display(RTB_dataset_df1['Attack_type'].value_counts().to_frame().style.set_caption("RTB_dataset_df1"))
display(RTB_dataset_df2['Attack_type'].value_counts().to_frame().style.set_caption("RTB_dataset_df2"))

,count
Attack_type,
Backdoor,81549
DDoS_HTTP,62613
Password,62373
DDoS_TCP,60349
DDoS_ICMP,46182
DDoS_UDP,3258


,count
Attack_type,
MITM,99812
Fingerprinting,99745
Ransomware,94039
XSS,88183
Port_Scanning,84733
Uploading,70749
SQL_injection,61693
Vulnerability_scanner,61128


In [87]:
# Concatenate RTB_dataset_df1 and RTB_dataset_df2
RTB_dataset = pd.concat([RTB_dataset_df1, RTB_dataset_df2], ignore_index=True)
del RTB_dataset_df1, RTB_dataset_df2

# Save RTB_dataset to csv file
RTB_dataset.to_csv("../data/RTB_dataset.csv", index=False)

In [88]:
# Count the number of rows per Attack_type in RTB_dataset
counts_dict_rtb = {}
for attack in RTB_dataset["Attack_type"].unique():
    counts_dict_rtb[attack] = RTB_dataset[RTB_dataset["Attack_type"] == attack].shape[0]
    
# Count the number of rows per Attack_type in df_train
counts_dict_train = {}
for attack in df_train["Attack_type"].unique():
    counts_dict_train[attack] = df_train[df_train["Attack_type"] == attack].shape[0]

counts_dict_total = {}
# Sum if key exists in both dictionaries
for key in counts_dict_rtb.keys() & counts_dict_train.keys():
    counts_dict_total[key] = counts_dict_rtb[key] + counts_dict_train[key]

counts_excess = {}
# Subtract 100.000 to get the number of rows to be removed
for key, value in counts_dict_total.items():
    counts_excess[key] = value - 100000

counts_excess

{'Backdoor': 793,
 'Ransomware': 1802,
 'DDoS_HTTP': 1529,
 'Password': 2198,
 'MITM': 94,
 'XSS': 214,
 'Port_Scanning': 648,
 'Uploading': 283,
 'DDoS_TCP': 402,
 'Fingerprinting': 452,
 'SQL_injection': 2448,
 'DDoS_UDP': 224,
 'DDoS_ICMP': 620,
 'Vulnerability_scanner': 1214}

In [90]:
# Remove excess rows from RTB_dataset by selecting a random sample of rows to be removed
for key, value in counts_excess.items():
    if value > 0:
        RTB_dataset = RTB_dataset.drop(RTB_dataset[RTB_dataset["Attack_type"] == key].sample(n=value, random_state=42).index)

# Concatenate RTB_dataset and df_train
df_balanced = pd.concat([df_train, RTB_dataset], ignore_index=True)

display(df_balanced['Attack_type'].value_counts().to_frame().style.set_properties(**{'text-align': 'left'}))

,count
Attack_type,
DDoS_UDP,100000
Password,100000
DDoS_TCP,100000
Backdoor,100000
DDoS_ICMP,100000
Port_Scanning,100000
Vulnerability_scanner,100000
SQL_injection,100000
DDoS_HTTP,100000


In [91]:
# Save df_balanced to csv file
df_balanced.to_csv("../data/EdgeIIot_train_100k_RTB_balanced.csv", index=False)

In [92]:
# Fill Attack_label column with 1 if Attack_type is not Normal, 0 otherwise
df_balanced["Attack_label"] = df_balanced["Attack_type"].apply(lambda x: 1 if x != "Normal" else 0)

In [98]:
display(df_balanced.describe().T.style.set_properties(**{'text-align': 'left'}))

,count,mean,std,min,25%,50%,75%,max
arp.opcode,1500000.000000,0.000021,0.006000,0.000000,0.000000,0.000000,0.000000,2.000000
arp.hw.size,1500000.000000,0.000084,0.022450,0.000000,0.000000,0.000000,0.000000,6.000000
icmp.checksum,1500000.000000,3824.459987,11903.002549,0.000000,0.000000,0.000000,0.000000,65533.000000
icmp.seq_le,1500000.000000,2379.180167,9827.704760,0.000000,0.000000,0.000000,0.000000,65534.000000
icmp.unused,1500000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
http.content_length,1500000.000000,13.561247,104.671315,0.000000,0.000000,0.000000,0.000000,83655.000000
http.response,1500000.000000,0.042481,0.201685,0.000000,0.000000,0.000000,0.000000,1.000000
http.tls_port,1500000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
tcp.ack,1500000.000000,72415358.421074,305137710.739505,0.000000,0.000000,1.000000,486.000000,3178709301.000000
tcp.ack_raw,1500000.000000,1273166826.755494,1280284214.791586,0.000000,0.000000,997129949.000000,2296324424.000000,4294926648.000000


: 